# Proof of Concept - Rolling Codes in FOB for Educational Purpose

## Idea
The project aims to create a basic Python-based rolling code system, serving as an educational tool to demonstrate the functionality of remote devices, such as car unlock systems. This implementation is simplified and not intended for real-world use but offers a clear, conceptual understanding of the technology.

## Notes
This is neither a secure nor a functional solution for practical applications. The development is loosely inspired by BMW's E-Series mechanics and a simplified version of the KeeLoq system. To aid understanding, certain complexities are reduced. For instance, data storage and transmission are handled using JSON payloads, as opposed to bit-level operations typical in actual devices.

## How it works

<b>Key Points:</b>
- The sender (key fob) only transmits data and does not receive any. This limits the encryption strength.
- Typically, the sender lacks real-time clocks, pseudorandom number generators, and other features necessary for robust encryption.

<b>Workflow:</b>
1. Pairing: The fob pairs with the receiver (e.g., a car) usually initiated by a button sequence on the receiver. The fob sends its ID and a seed value.
2. Code Generation: Both the sender and receiver generate a list of rolling codes using the sender's public ID and the private seed. This allows for different, synchronized code lists for each sender, enabling multiple fobs per receiver.
3. Transmission: When activated, the fob sends a payload containing its ID, a code from the list (selected based on an internal counter), and sometimes the button state. It then increments its counter and generates a new code to ensure an ongoing supply.
4. Reception and Verification: The receiver processes the payload, using its own counter to retrieve the corresponding code. If the codes match, the receiver increments its counter.
5. Sync Issues: If a fob is used out of the receiver's range, its counter advances more than the receiver's, causing a mismatch. The receiver has a threshold for checking subsequent codes but never checks past codes. If a match is found within this threshold, it re-syncs by adjusting its counter. Otherwise, re-pairing is necessary.


## Possible Attacks

### Replay Attack
An attacker intercepts (man-in-the-middle attack) and saves a rolling code, attempting to reuse it for unauthorized access. This is generally thwarted by the ever-increasing counter, which doesn't decrease. However, some systems remain vulnerable, as evidenced in certain cases (e.g., Honda key fob vulnerability, detailed here: https://securityintelligence.com/articles/what-to-know-honda-key-fob-vulnerability/).

### Rolljam Attack
This more sophisticated attack involves jamming and recording the first transmission, misleading the user to retransmit. The attacker then uses the first code immediately and reserves the second for future unauthorized access. More details are available in Samy Kamkar's DEF CON presentation (https://samy.pl/defcon2015/).

<b>Workflow:</b>
1. A user activates the fob button, initiating the first signal. An attacker then disrupts and captures this signal, leading to no transmission to and no response from the receiver.
2. Assuming a malfunction or range issue with the fob, the user typically attempts to use the fob again. During this second attempt, the attacker also interferes and records the signal, thus obtaining two valid rolling codes.
3. The attacker promptly transmits the first recorded code to the receiver. The receiver responds this time, leading the user to believe the second attempt was successful and the initial failure was a mere glitch.
4. Unbeknownst to the user, the attacker retains the second valid code, which remains effective for unauthorized access later, provided there are no subsequent transmissions from the fob.

<b>Technical Explaination:</b>
From a technical standpoint, suppose the receiver's counter is at a value like 61. When the user presses the fob button, the fob transmits codes 61 and 62 successively, but both signals are intercepted by the attacker. The attacker then sends code 61 to the receiver, which the receiver accepts as valid, advancing its counter to 62. As a result, the attacker retains the code 62, which remains valid for unauthorized use since the receiver's counter matches or is less than 62, due to the earlier signal interference.


## Demo cases in this repository
1. Pairing multiple devices with different seeds.
2. Sending a valid request (e.g., pressing the fob button).
3. Sending a valid request with an out-of-sync counter.
4. Sending an invalid request due to significant counter mismatch.
5. Replay attack demonstration.
6. Rolljam attack scenario.


## Resources
For more detailed technical information, refer to the KeeLoq documentation available: https://ww1.microchip.com/downloads/en/AppNotes/91002a.pdf

In [10]:
from Crypto.Hash import SHA256

### Settings ###
DEBUG = True

In [11]:
class Receiver:
    def __init__(self):
        self.receiver = {}
        
    def clear_receiver(self):
        self.receiver = {}
        
    def pair_new_device(self, pairing_payload):
        if pairing_payload['Id'] not in self.receiver:
            self.receiver[pairing_payload['Id']] = {
                'Count' : 0,
                'Codes' : self.generate_rolling_code_list(0, 10, pairing_payload['Id'], pairing_payload['Seed']),
                'Seed' : pairing_payload['Seed']
            }
            if DEBUG: print('Device paired')
        return self.receiver

    def generate_rolling_code_list(self, start_code, list_len, sender_id, seed):
        rolling_codes = []
        for i in range (start_code, list_len):
            hash = SHA256.new()
            hash.update(
                (sender_id + seed + str(i)).encode('utf-8')
            )
            rolling_codes.append(
                hash.hexdigest()
            )
        return rolling_codes

    def check_sender_id_in_allowlist(self, sender_id):
        if sender_id in self.receiver:
            if DEBUG: print('Sender is paired with receiver')
            return True
        else:
            raise Exception("Invalid Sender Id") 

    def check_rolling_code(self, payload):
        sender_count = self.receiver[payload['Id']]['Count'] 
        while True:
            try:
                receiver_code = self.receiver[payload['Id']]['Codes'][sender_count]
            except:
                raise Exception("FOB out of sync") 
            sender_count += 1
            if payload['Code'] == receiver_code:
                if DEBUG: print('Code valid')
                return sender_count
            print('Counter out of sync. Increase counter')

    def set_new_receiver_code_count(self, sender_id, count):
        self.receiver[payload['Id']]['Count'] = count
        if DEBUG: print('Receiver code count adjusted')

    def validate_payload_adjust_codes(self, payload):
        self.check_sender_id_in_allowlist(
            payload['Id']
        )

        sender_count_new = self.check_rolling_code(
            payload
        )
        
        self.receiver[payload['Id']]['Codes'].extend(
            self.generate_rolling_code_list(
                self.receiver[payload['Id']]['Count'] + 10, 
                (sender_count_new + 10), 
                payload['Id'], 
                self.receiver[payload['Id']]['Seed']
            )
        )

        self.set_new_receiver_code_count(
            payload['Id'], 
            sender_count_new
        )
        
        if DEBUG: print('Extended code list')

In [12]:
class Sender:
    def __init__(self):
        fob = {}
        
    def set_fob_data(self, fob):
        self.fob = fob
        
    def create_pairing_payload(self):
        return {
            'Id' : self.fob['Id'],
            'Seed' : self.fob['Seed']
        }
        
    def create_send_payload(self):
        return {
            'Id' : self.fob['Id'],
            'Code' : self.fob['Codes'][self.fob['Counter']]
        }
    
    def generate_rolling_code_list(self, start_code, list_len, sender_id, seed):
        rolling_codes = []
        for i in range (start_code, list_len):  
            hash = SHA256.new()
            hash.update(
                (sender_id + seed + str(i)).encode('utf-8')
            )
            rolling_codes.append(
                hash.hexdigest()
            )
        return rolling_codes
    
    def higher_counter(self):
        self.fob['Counter'] += 1
        return self.fob
    
    def extend_rolling_code(self):
        self.fob['Codes'].extend(
            self.generate_rolling_code_list(
                self.fob['Counter'] + 10, 
                (self.fob['Counter'] + 1 + 10), 
                self.fob['Id'], 
                self.fob['Seed']
            )
        )

In [13]:
### Pairing of two FOBs ###

#Note: The payloads are for educational purpose in JSON format. Usually it will be saved in bits

### Initial State of the devices ###
def pair_devices():
    receiver = Receiver()
    fob_1 = Sender()
    fob_2 = Sender()

    fob_1.set_fob_data(
        {
            'Id' : '213u123z1281',
            'Seed' : '7827637huasz23',
            'Counter' : 0,
            'Codes' : fob_1.generate_rolling_code_list(0, 10, '213u123z1281', '7827637huasz23'),
        }
    )

    fob_2.set_fob_data(
        {
            'Id' : '78923491823s',
            'Seed' : '3217676128hsa12',
            'Counter' : 0,
            'Codes' : fob_2.generate_rolling_code_list(0, 10, '78923491823s', '3217676128hsa12'),
        }
    )


    ### Payload from FOB (always sender, never receives) ###
    pairing_payload_fob_1 = fob_1.create_pairing_payload()
    pairing_payload_fob_2 = fob_2.create_pairing_payload()

    ### Mock a pairing process ###

    #Usually you will start the pairing process by doing a combination of ignition state and button pressing
    print('Pairing started ...')

    #At least in BMW cars all paired devices get removed after starting the process
    print('Clear all paired devices ...')
    receiver.clear_receiver()

    #Usually you will send a payload by pressing multiple buttons the same time at the FOB
    for payload in [pairing_payload_fob_1, pairing_payload_fob_2]:
        receiver.pair_new_device(payload)
        print('Waiting 10 seconds for other pairing')

    #Pairing ends if no new device is paired in a short timeframe or after turning off ignition
    print('Pairing done through timeout or power off')
    
    return [receiver, fob_1, fob_2]
    
receiver, fob_1, fob_2 = pair_devices()

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off


In [14]:
### Case: Sender sends valid code to receiver ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

print('Done')

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
Done


In [15]:
### Case: Sender2 sends code then Sender1 sends code ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

#Now the same for fob_2
print(10*'-')
payload = fob_2.create_send_payload()
receiver.validate_payload_adjust_codes(payload)
fob_2.extend_rolling_code()
fob_2.higher_counter()

print('Done')

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
----------
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
Done


In [16]:
### Case: Sender sends code <10 away from last code (FOB out of reach) ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#FOB higher counter to mock multiple button pressing without range while receiver is on old counter
for i in range(0, 8):
    fob_1.higher_counter()
    fob_1.extend_rolling_code()
payload = fob_1.create_send_payload()

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

print('Done')

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Code valid
Receiver code count adjusted
Extended code list
Done


In [17]:
### Case: Sender sends code >10 away from last code (FOB out of reach) ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#FOB higher counter to mock multiple button pressing without range while receiver is on old counter
for i in range(0, 20):
    fob_1.higher_counter()
    fob_1.extend_rolling_code()
payload = fob_1.create_send_payload()

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

print('Done')

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter


Exception: FOB out of sync

In [18]:
### Case: Sender replays old code - Replay Attack ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

payload_saved = payload

#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

#Replaying an already sent payload
receiver.validate_payload_adjust_codes(payload_saved)

#FOB lose sync because the payload is valid but not in sync with the increment counter

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
Sender is paired with receiver
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter
Counter out of sync. Increase counter


Exception: FOB out of sync

In [19]:
### Case: RollJam Attack ###

#Pair devices for clean start
receiver, fob_1, fob_2 = pair_devices()

#FOB creates Payload
payload = fob_1.create_send_payload()

#An attacker now jams the transmission but captures the key
payload_saved_1 = payload

#The key still generates a new key and higher counter
fob_1.higher_counter()
fob_1.extend_rolling_code()

#The user sees that the car doesn't locks and press the button again. 
payload = fob_1.create_send_payload()

#The attacker now jams it again and captures it
payload_saved_2 = payload

#Now the attacker sends first captured payload to give the user a positive feedback (car locks)
#Receiver validates Payload, higher count after success and generate a new code at end of the list
receiver.validate_payload_adjust_codes(payload_saved_1)

#FOB extend a new code at end of the list
fob_1.extend_rolling_code()

#FOB highers counter after each transmission
fob_1.higher_counter()

#The attacker has now an unsed key which is still in sync with the counter in receiver (because jammed before)
#The attacker can send this payload at a later time and gains control
receiver.validate_payload_adjust_codes(payload_saved_2)

Pairing started ...
Clear all paired devices ...
Device paired
Waiting 10 seconds for other pairing
Device paired
Waiting 10 seconds for other pairing
Pairing done through timeout or power off
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
Sender is paired with receiver
Code valid
Receiver code count adjusted
Extended code list
